In [1]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.system_object_type import FileSystemObjectType
import os
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )

In [2]:
#checking the process on the base url and relative url
site_url = "https://hksinc.sharepoint.com/sites/SpatialAnalyticsModule"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()

In [8]:
def downloadfile(file_url):
    # Create a directory to store the text files
    if not os.path.exists("pdf/"):
            os.mkdir("pdf/")

    if not os.path.exists("pdf/hksinc.sharepoint.com/"):
            os.mkdir("pdf/hksinc.sharepoint.com/")

    download_path = os.path.join("pdf/hksinc.sharepoint.com", os.path.basename(file_url))
    with open(download_path, "wb") as local_file:
        file = ctx.web.get_file_by_server_relative_url(file_url).download(local_file).execute_query()
        #file = ctx.web.get_file_by_server_relative_url(file_url).read()
    print("[Ok] file has been downloaded into: {0}".format(download_path))

In [9]:
#documents on current site url
doc_lib = ctx.web.lists.get_by_title("Documents")
items = doc_lib.items.select(["FileSystemObjectType"]).expand(["File", "Folder"]).get().execute_query()
for item in items:  # type: ListItem
    
    if item.file_system_object_type == FileSystemObjectType.Folder:
        print("(Folder): {0}".format(item.folder.serverRelativeUrl))
    else:#pdf only
        if("pdf" in item.file.serverRelativeUrl):
            print("(File): {0}".format(item.file.serverRelativeUrl))
            downloadfile(item.file.serverRelativeUrl)

(File): /sites/SpatialAnalyticsModule/Shared Documents/02_PhotoEssay_On-Site Documentation_Example.pdf


ValueError: 

In [7]:
# importing required modules
from PyPDF2 import PdfReader

def read_pdf(file):
    # creating a pdf reader object
    reader = PdfReader('pdf/hksinc.sharepoint.com/05_Interviews_Documentation Example.pdf')
    
    # printing number of pages in pdf file
    print(len(reader.pages))
    
    # getting a specific page from the pdf file
    page = reader.pages[0]
    
    # extracting text from page
    text = page.extract_text()
    print(text)

2
INTERVIEWS  
 
Methodology:  
Semi -structured interviews were conducted during the design diagnostic with selected staff who work in 
the Sengkang Hospital Emergency Department  (ED) . The interviews included multiple open -ended  
questions that focused on : 1) design elements in the ED  that support staff work practices (facilitators), 2) 
design elements in the ED  that hinder staff work practices (barriers), and 3) operational considerations 
for moving forward.  Each interview was co nducted face to face and lasted between  10 – 15 minutes . 
Notes were taken during the interviews and later synthesized  to identify  environmental facilitators and 
barriers  and operational considerations , as well as current state and future state design and o perational 
recommendations.  
 Participants:  
Facilities Manager  
P1 Mid -level Physician  
P2 Senior Physician  
P2 Junior Staff Nurse  
 Demographics:  
A total of 4  interviews were conducted with staff including facilities manageme